In [2]:
import sys, os

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from debugger.bench_viewer import BenchViewer
from debugger.strainer import DebugStatus
from benchmark_consts import *
from debugger.mutant_info import TraceFailure
from debugger.debugger import DbgMode, DebugOptions

In [3]:
verus_rd2_to_targets = [
    "621abdd996",
    "aad03b9358",
    "199ba4594c",
    "93004876e4",
    "65d90190b4",
    "fa2fdaf6e0",
    "6937ca47cd",
    "82449bd2e8",
    "a1b4b3fd4a",
    "8c0bbf67d0",
    "55b4af82a4",
    "66c9bdf69f",
]

options = DebugOptions()
options.mode = DbgMode.DOUBLETON
options.is_verus = False

mv = BenchViewer(verus_rd2_to_targets, options)

[WARN] [init] data/projs/verismo.dedup/base.z3/allocator__linkedlist.10.smt2 detected IS Verus, differ from the given option [WARN] [init] data/projs/verismo.dedup/base.z3/pgtable_e__pte.18.smt2 detected IS Verus, differ from the given option [WARN] [init] data/projs/verismo.dedup/base.z3/boot__init__e820_init.2.smt2 detected IS Verus, differ from the given option [WARN] [init] data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__liveness__stateful_set_match.2.smt2 detected IS Verus, differ from the given option [WARN] [init] data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__resource_match__lemma_from_after_get_resource_step_to_after_update_resource_step.smt2 detected IS Verus, differ from the given option [WARN] [init] data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__resource_match.1.smt2 detected IS Verus, differ from the given option [WARN] [init] data/projs/flowcert/base.z3/permission.14

In [4]:
mv.status.print_status()

| category                  |   count | percentage   |
|---------------------------|---------|--------------|
| DebugStatus.FIX_NOT_FOUND |       8 | 66.67 %      |
| DebugStatus.FIX_FOUND     |       4 | 33.33 %      |
| total                     |      12 | 100.00 %     |


In [5]:
for q in mv.status[DebugStatus.NO_PROOF]:
    print(q)

In [6]:
for q in mv.status[DebugStatus.ERROR]:
    print("./src/debugger3.py --verus --mode doubleton --reset -i",  mv[q].name_hash)

In [7]:
for q in mv.status[DebugStatus.NOT_CREATED]:
    print("./src/debugger3.py --verus --mode doubleton --create -i", mv[q].name_hash)

In [ ]:
from utils.system_utils import list_smt2_files

# for q in mv.status[DebuggerStatus.FILTERED_NOT_RAN]:

for q in mv.status[DebugStatus.NOT_TESTED]:
    count = len(list_smt2_files(mv[q].strainer.test_dir))
    # assert 0 < count <= 10
    # print("./src/debugger3.py --mode doubleton --create -i", mv[q].name_hash)
    print("./src/make_spaghet.py --cluster --verus -i", mv[q].strainer.test_dir)

In [11]:
from utils.cluster_utils import get_sync_commands
from utils.system_utils import write_misc_script
dirs = []
for q in mv.status[DebugStatus.FIX_FOUND]:
    dbg = mv[q]
    # if len(dbg.report.tested) > 10:
    #     print(q)
    dirs += dbg.get_sync_dirs()

# get_sync_commands("g2001", dirs)
write_misc_script("sync.sh", get_sync_commands("g2001", dirs))

[INFO] created misc script: sync.sh with 24 commands 
